# For using the Roadsign Classifier Web App, go to the following link : https://59bc-34-74-152-238.ngrok-free.app/

If the link changes, just go to the link generated by the forth code block, which is the same code block above the next main header. For running the code on a new Google Colab session, you need to create a free account with ngrok and add your authtoken. If Running in Google Colab, create a secret with the name 'ngrok' with the value as your authtoken. For updating the code on the server, run all the code blocks

In [15]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install streamlit
!pip install pyngrok


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [19]:
from google.colab import userdata
secret = userdata.get('ngrok')
!ngrok authtoken $secret # MUST UNCOMMENT LINE AND ADD AUTHTOKEN TO RUN FRONT END

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [17]:
%%writefile app.py
### Current Iteration of Model ###

### How to Use Model
### 1. Upload .png of roadsign to colab file storage
### 2. change image_path variable to be the name of the roadsign file
# Update the image path to your image
import cv2
import numpy as np
import pytesseract
from google.colab.patches import cv2_imshow  # Use this to display images in Colab
import streamlit as st
from PIL import Image

# Ensure Tesseract is properly set up
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'  # Update if necessary

def detect_road_sign(image):
    original_height, original_width = image.shape[:2]

    # Convert to BGR if the input is in RGB
    image = cv2.cvtColor(image, cv2.COLOR_RGBA2BGR)

    # Resize the image for better processing
    image_resized = cv2.resize(image, (600, 400))

    # Convert to grayscale
    gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Perform edge detection using Canny
    edged = cv2.Canny(blurred, 50, 150)

    # Convert to HSV
    hsv = cv2.cvtColor(image_resized, cv2.COLOR_BGR2HSV)

    # Define color ranges
    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 70, 50])
    upper_red2 = np.array([180, 255, 255])
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])

    # Threshold the HSV image for colors
    mask_red = cv2.bitwise_or(cv2.inRange(hsv, lower_red1, upper_red1),
                              cv2.inRange(hsv, lower_red2, upper_red2))
    mask_white = cv2.inRange(hsv, lower_white, upper_white)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Clean up masks with morphological operations
    kernel = np.ones((3, 3), np.uint8)
    mask_red = cv2.morphologyEx(mask_red, cv2.MORPH_OPEN, kernel, iterations=2)
    mask_red = cv2.dilate(mask_red, kernel, iterations=1)
    mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_OPEN, kernel, iterations=2)
    mask_white = cv2.dilate(mask_white, kernel, iterations=1)
    mask_yellow = cv2.morphologyEx(mask_yellow, cv2.MORPH_OPEN, kernel, iterations=2)
    mask_yellow = cv2.dilate(mask_yellow, kernel, iterations=1)

    # Detect Stop Signs and No Parking Signs
    contours_red, _ = cv2.findContours(mask_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours_red:
        area = cv2.contourArea(contour)
        if area > 500:
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w) / h
            perimeter = cv2.arcLength(contour, True)
            circularity = 4 * np.pi * (area / (perimeter * perimeter))

            if circularity > 0.7:  # Circular shapes (for No Parking signs)
                roi = image_resized[y:y + h, x:x + w]
                roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                _, binary_roi = cv2.threshold(roi_gray, 50, 255, cv2.THRESH_BINARY_INV)
                custom_config = r'--psm 6'
                detected_text = pytesseract.image_to_string(binary_roi, config=custom_config)
                if 'P' in detected_text or "NO PARKING" in detected_text.upper():
                    cv2.rectangle(image_resized, (x, y), (x + w, y + h), (0, 255, 0), 3)
                    cv2.putText(image_resized, "No Parking Sign", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                elif len(approx) == 8 and 0.8 < aspect_ratio < 1.2:
                    cv2.drawContours(image_resized, [approx], 0, (0, 0, 255), 3)
                    cv2.putText(image_resized, "Stop Sign", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)



    # Detect Speed Limit Signs
    contours_white, _ = cv2.findContours(mask_white, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours_white:
        area = cv2.contourArea(contour)
        if area > 1000:
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w) / h
            if len(approx) == 4 and 0.5 < aspect_ratio < 1.5:
                cv2.drawContours(image_resized, [approx], 0, (255, 0, 0), 3)
                cv2.putText(image_resized, "Speed Limit Sign", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    # Detect Yield Signs
    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 800:
            epsilon = 0.04 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            if len(approx) == 3:
                x, y, w, h = cv2.boundingRect(approx)
                aspect_ratio = float(w) / h
                if 0.5 < aspect_ratio < 1.5:
                    cv2.drawContours(image_resized, [approx], 0, (0, 255, 255), 3)
                    cv2.putText(image_resized, "Yield Sign", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    # Detect Railroad Crossing Signs
    contours_yellow, _ = cv2.findContours(mask_yellow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # List to store bounding boxes for merging
    bounding_boxes = []

    for contour in contours_yellow:
        # Filter smaller contours to avoid noise
        area = cv2.contourArea(contour)
        if area > 500:  # Lowered the area threshold for detecting smaller or dim regions
            # Approximate the contour
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            # Check if the contour is roughly circular
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w) / h
            if 0.7 < aspect_ratio < 1.3:  # Allow a broader range for aspect ratio
                # Check the circularity
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * (area / (perimeter * perimeter))
                if circularity > 0.6:  # Lowered the circularity threshold for imperfect circles
                    bounding_boxes.append((x, y, w, h))

    # Merge bounding boxes into a single region
    if bounding_boxes:
        x_min = min([x for x, y, w, h in bounding_boxes])
        y_min = min([y for x, y, w, h in bounding_boxes])
        x_max = max([x + w for x, y, w, h in bounding_boxes])
        y_max = max([y + h for x, y, w, h in bounding_boxes])

        # Extract the merged bounding box
        roi = image_resized[y_min:y_max, x_min:x_max]
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        _, binary_roi = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY_INV)  # Lowered the binary threshold

        # Detect lines or 'X' shape in the ROI
        edges = cv2.Canny(binary_roi, 30, 120)  # Adjusted edge detection thresholds
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=25, minLineLength=20, maxLineGap=10)

        if lines is not None and len(lines) >= 4:  # Expect at least 4 lines for 'X'
            cv2.rectangle(image_resized, (x_min, y_min), (x_max, y_max), (0, 255, 0), 3)
            cv2.putText(image_resized, "Railroad Crossing", (x_min, y_min - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    image_resized = cv2.cvtColor(image_resized, cv2.COLOR_RGBA2BGR)
    image_resized = cv2.resize(image_resized, (original_width, original_height))

    return image_resized

# Streamlit GUI
st.title("Road Sign Detector")
st.write("By Austin McCormick")

uploaded_file = st.file_uploader("Choose a road sign image", type=["png", "jpg", "jpeg"])

if uploaded_file is not None:
    # Read the uploaded image
    image = Image.open(uploaded_file)
    image_np = np.array(image)  # Convert to numpy array

    # Process the image using the detection function
    processed_image = detect_road_sign(image_np)

    # Display the original and processed images
    col1, col2 = st.columns(2)
    with col1:
        st.image(image, caption="Uploaded Image", use_container_width=True)
    with col2:
        st.image(processed_image, caption="Processed Image", use_container_width=True)



Overwriting app.py


In [20]:
from pyngrok import ngrok
!streamlit run app.py &>/content/logs.txt &
public_url = ngrok.connect(8501, "http")  # Explicitly specify HTTP protocol

print(f"Streamlit app running at {public_url}")


Streamlit app running at NgrokTunnel: "https://a32ef50741f9.ngrok-free.app" -> "http://localhost:8501"


# **ALL CODE BELOW HERE IS PAST ITERATIONS AND TEST CODE**

In [ ]:
'''
import cv2
import numpy as np
import pytesseract
from google.colab.patches import cv2_imshow  # Use this to display images in Colab

# Ensure Tesseract is properly set up
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'  # Update if necessary

def detect_road_sign(image_path):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Unable to load the image.")
        return

    # Resize the image for better processing
    image_resized = cv2.resize(image, (600, 400))

    # Convert to grayscale
    gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Perform edge detection using Canny
    edged = cv2.Canny(blurred, 50, 150)

    # Convert to HSV
    hsv = cv2.cvtColor(image_resized, cv2.COLOR_BGR2HSV)

    # Define color ranges
    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 70, 50])
    upper_red2 = np.array([180, 255, 255])
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])

    # Threshold the HSV image for colors
    mask_red = cv2.bitwise_or(cv2.inRange(hsv, lower_red1, upper_red1),
                              cv2.inRange(hsv, lower_red2, upper_red2))
    mask_white = cv2.inRange(hsv, lower_white, upper_white)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Clean up masks with morphological operations
    kernel = np.ones((3, 3), np.uint8)
    mask_red = cv2.morphologyEx(mask_red, cv2.MORPH_OPEN, kernel, iterations=2)
    mask_red = cv2.dilate(mask_red, kernel, iterations=1)
    mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_OPEN, kernel, iterations=2)
    mask_white = cv2.dilate(mask_white, kernel, iterations=1)
    mask_yellow = cv2.morphologyEx(mask_yellow, cv2.MORPH_OPEN, kernel, iterations=2)
    mask_yellow = cv2.dilate(mask_yellow, kernel, iterations=1)

    # Detect Stop Signs and No Parking Signs
    contours_red, _ = cv2.findContours(mask_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours_red:
        area = cv2.contourArea(contour)
        if area > 500:
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w) / h
            perimeter = cv2.arcLength(contour, True)
            circularity = 4 * np.pi * (area / (perimeter * perimeter))

            if circularity > 0.7:  # Circular shapes (for No Parking signs)
                roi = image_resized[y:y + h, x:x + w]
                roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                _, binary_roi = cv2.threshold(roi_gray, 50, 255, cv2.THRESH_BINARY_INV)
                custom_config = r'--psm 6'
                detected_text = pytesseract.image_to_string(binary_roi, config=custom_config)
                if 'P' in detected_text or "NO PARKING" in detected_text.upper():
                    cv2.rectangle(image_resized, (x, y), (x + w, y + h), (0, 255, 0), 3)
                    cv2.putText(image_resized, "No Parking Sign", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                elif len(approx) == 8 and 0.8 < aspect_ratio < 1.2:
                    cv2.drawContours(image_resized, [approx], 0, (0, 0, 255), 3)
                    cv2.putText(image_resized, "Stop Sign", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)



    # Detect Speed Limit Signs
    contours_white, _ = cv2.findContours(mask_white, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours_white:
        area = cv2.contourArea(contour)
        if area > 1000:
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w) / h
            if len(approx) == 4 and 0.5 < aspect_ratio < 1.5:
                cv2.drawContours(image_resized, [approx], 0, (255, 0, 0), 3)
                cv2.putText(image_resized, "Speed Limit Sign", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    # Detect Yield Signs
    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 800:
            epsilon = 0.04 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            if len(approx) == 3:
                x, y, w, h = cv2.boundingRect(approx)
                aspect_ratio = float(w) / h
                if 0.5 < aspect_ratio < 1.5:
                    cv2.drawContours(image_resized, [approx], 0, (0, 255, 255), 3)
                    cv2.putText(image_resized, "Yield Sign", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    # Detect Railroad Crossing Signs
    contours_yellow, _ = cv2.findContours(mask_yellow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # List to store bounding boxes for merging
    bounding_boxes = []

    for contour in contours_yellow:
        # Filter smaller contours to avoid noise
        area = cv2.contourArea(contour)
        if area > 500:  # Lowered the area threshold for detecting smaller or dim regions
            # Approximate the contour
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            # Check if the contour is roughly circular
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w) / h
            if 0.7 < aspect_ratio < 1.3:  # Allow a broader range for aspect ratio
                # Check the circularity
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * (area / (perimeter * perimeter))
                if circularity > 0.6:  # Lowered the circularity threshold for imperfect circles
                    bounding_boxes.append((x, y, w, h))

    # Merge bounding boxes into a single region
    if bounding_boxes:
        x_min = min([x for x, y, w, h in bounding_boxes])
        y_min = min([y for x, y, w, h in bounding_boxes])
        x_max = max([x + w for x, y, w, h in bounding_boxes])
        y_max = max([y + h for x, y, w, h in bounding_boxes])

        # Extract the merged bounding box
        roi = image_resized[y_min:y_max, x_min:x_max]
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        _, binary_roi = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY_INV)  # Lowered the binary threshold

        # Detect lines or 'X' shape in the ROI
        edges = cv2.Canny(binary_roi, 30, 120)  # Adjusted edge detection thresholds
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=25, minLineLength=20, maxLineGap=10)

        if lines is not None and len(lines) >= 4:  # Expect at least 4 lines for 'X'
            cv2.rectangle(image_resized, (x_min, y_min), (x_max, y_max), (0, 255, 0), 3)
            cv2.putText(image_resized, "Railroad Crossing", (x_min, y_min - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Display the output
    cv2_imshow(image_resized)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Update the image path
image_path = "speedlimit_real.PNG"
detect_road_sign(image_path)
'''

In [ ]:
'''
### Sketch Roadsign Detection for Stop signs, Yield signs, and Speedlimit signs ###
### Code for real roadsign detection from images further below ###
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pytesseract
import os
from google.colab.patches import cv2_imshow

def remove_writing(img_path):
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Binarize the image using thresholding
        _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        filled_image = img.copy()

        for contour in contours:
            if cv2.contourArea(contour) > 500:
                # Dilate the contour area slightly to expand the filled area
                mask = np.zeros_like(binary)
                cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)
                dilated_mask = cv2.dilate(mask, np.ones((3, 3), np.uint8), iterations=1)

                # Fill the dilated contour with white
                filled_image[dilated_mask == 255] = [255, 255, 255]

                # Redraw the contour outline with a thick line to reinforce the border
                cv2.drawContours(filled_image, [contour], -1, (0, 0, 0), thickness=2)
        return filled_image

class HeuristicRoadSignRecognizer:
    def __init__(self):
        pass

    def recognize_sign(self, image_path):
        # Check if the file exists
        if not os.path.exists(image_path):
            print(f"Error: The file '{image_path}' was not found.")
            return "File not found", {}

        # Load the image in grayscale

        img = remove_writing(image_path)

        # Check if the image was loaded correctly
        if img is None:
            print(f"Error: Unable to load image '{image_path}'. Please check the file format.")
            return "Image load error", {}

        # Preprocessing: Apply binary threshold
        _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

        # Apply Gaussian blur to smooth edges
        img = cv2.GaussianBlur(img, (5, 5), 0)

        # Apply morphological operations to remove small noise
        kernel = np.ones((3, 3), np.uint8)
        img = cv2.erode(img, kernel, iterations=1)
        img = cv2.dilate(img, kernel, iterations=1)
        if len(img.shape) == 3:  # Check if image has 3 channels
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Display the original image
        original_img = cv2.imread(image_path)  # Load the original image
        original_img_rgb = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        plt.imshow(original_img_rgb)
        plt.title("Original Sketch:")
        plt.axis("off")
        plt.show()

        # Display the preprocessed image
        plt.imshow(img, cmap="gray")
        plt.title("Preprocessed Sketch for Corner Detection:")
        plt.axis("off")
        plt.show()

        # Calculate heuristic features
        features = {}

        # Feature 1: Aspect Ratio
        height, width = img.shape
        features["aspect_ratio"] = width / height

        # Feature 2: Fill Density (number of black pixels)
        num_black_pixels = np.sum(img == 0)
        features["fill_density"] = num_black_pixels / (height * width)

        # Feature 3: Symmetry
        half_width = width // 2
        vertical_symmetry = np.sum(img[:, :half_width] == np.fliplr(img[:, width - half_width:])) / (height * half_width)
        horizontal_symmetry = np.sum(img[:height // 2, :] == np.flipud(img[height - height // 2:, :])) / ((height // 2) * width)
        features["vertical_symmetry"] = vertical_symmetry
        features["horizontal_symmetry"] = horizontal_symmetry

        # Feature 4: Corner Detection
        corners_harris = cv2.goodFeaturesToTrack(img, maxCorners=50, qualityLevel=0.01, minDistance=400, useHarrisDetector=True, k=0.1)
        num_corners_harris = len(corners_harris) if corners_harris is not None else 0
        features["num_corners"] = num_corners_harris
        corners_display = np.intp(corners_harris)

        # Create a colored copy of the grayscale image for corner visualization
        color_image = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

        # Iterate through detected corners
        if corners_harris is not None:
            corners_display = np.intp(corners_harris)
            for c in corners_display:
                x, y = c.ravel()
                # Highlight corners in a different color (e.g., red)
                cv2.circle(color_image, center=(x, y), radius=10, color=(0, 0, 255), thickness=-1)

        # Display the image with corners
        plt.imshow(cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB))
        plt.title("Detected Corner Locations:")
        plt.axis("off")
        plt.show()

        # Feature 5: Edge Density
        edges = cv2.Canny(img, 100, 200)
        edge_density = np.sum(edges == 255) / (height * width)
        features["edge_density"] = edge_density

        # Feature 6: Orientation Detection (for distinguishing triangle orientation)
        contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(contours) > 0:
            # Get the largest contour assuming it's the sign shape
            largest_contour = max(contours, key=cv2.contourArea)
            M = cv2.moments(largest_contour)
            if M["m00"] != 0:
                centroid_y = int(M["m01"] / M["m00"])
                x, y, w, h = cv2.boundingRect(largest_contour)
                bounding_box_center_y = y + h / 2

                # Determine if the centroid is in the upper or lower half of the bounding box
                if centroid_y < bounding_box_center_y:
                    orientation = "inverted"  # Likely a yield sign (inverted triangle)
                else:
                    orientation = "upright"  # Likely a warning sign (upright triangle)
            else:
                orientation = "unknown"
        else:
            orientation = "unknown"

        features["orientation"] = orientation

        # Feature 7: Line Detection (Presence of horizontal/vertical lines)
        num_horizontal_lines = 0
        num_vertical_lines = 0
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=30, maxLineGap=10)
        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                if abs(y2 - y1) < 5:  # Horizontal line
                    num_horizontal_lines += 1
                elif abs(x2 - x1) < 5:  # Vertical line
                    num_vertical_lines += 1
        features["horizontal_lines"] = num_horizontal_lines
        features["vertical_lines"] = num_vertical_lines

        # Feature 8: Proportion of Filled Area (Bounding Box)
        bounding_box_area = w * h
        features["filled_area_proportion"] = num_black_pixels / bounding_box_area

        # Recognize based on heuristic rules
        if features["num_corners"] == 3:
            if features["orientation"] == "upright":
                sign_type = "Warning (Triangle)"
            elif features["orientation"] == "inverted":
                sign_type = "Yield Sign"
            else:
                sign_type = "Unknown Triangle Sign"
        elif features["num_corners"] == 8:
            sign_type = "Stop Sign"
        #elif features["fill_density"] > 0.5 and features["num_corners"] == 0:
        elif features["num_corners"] < 3:
            sign_type = "Speed Limit Sign"
        else:
            sign_type = "Unknown Sign"

        return sign_type, features

# Example usage
recognizer = HeuristicRoadSignRecognizer()
sign_type, extracted_features = recognizer.recognize_sign("yield3.png")
print("Recognized sign type:", sign_type)
print("\nExtracted features:", extracted_features)
'''

In [ ]:
'''
### Test code to clean out sign interiors for preprocessing ###

import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Load the image
img = cv2.imread('yield3.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Binarize the image using thresholding
_, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

# Detect contours
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a copy of the original image to draw filled shapes and highlights
filled_image = img.copy()

# Loop over each detected contour
for contour in contours:
    # Optionally, filter contours by area to avoid small noise
    if cv2.contourArea(contour) > 500:  # Adjust the threshold as needed
        # Dilate the contour area slightly to expand the filled area
        mask = np.zeros_like(binary)
        cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)
        dilated_mask = cv2.dilate(mask, np.ones((3, 3), np.uint8), iterations=1)

        # Fill the dilated contour with white on the main image
        filled_image[dilated_mask == 255] = [255, 255, 255]

        # Redraw the contour outline with a thick line to reinforce the border
        cv2.drawContours(filled_image, [contour], -1, (0, 0, 0), thickness=2)  # Black border



# Display the result
cv2_imshow(filled_image)
'''

In [ ]:
'''
### test code, ML approach that uses cascade classifier xml files ###
import cv2
from google.colab.patches import cv2_imshow  # For displaying images in Google Colab

# Load the Stop Sign Cascade Classifier XML
stop_sign = cv2.CascadeClassifier('cascade_stop_sign.xml')

def detect_stop_sign(image_path):
    # Load the image
    img = cv2.imread(image_path)

    if img is None:
        raise FileNotFoundError(f"Could not load the image at path: {image_path}")

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect the stop sign
    stop_sign_scaled = stop_sign.detectMultiScale(gray, 1.3, 5)

    # Draw rectangles around detected stop signs and annotate
    for (x, y, w, h) in stop_sign_scaled:
        # Draw rectangle around the stop sign
        stop_sign_rectangle = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 3)
        # Write "Stop Sign" below the rectangle
        cv2.putText(img=stop_sign_rectangle,
                    text="Stop Sign",
                    org=(x, y + h + 30),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1,
                    color=(0, 0, 255),
                    thickness=2,
                    lineType=cv2.LINE_4)

    # Display the result
    cv2_imshow(img)

# Example usage
uploaded_image_path = "stopsign_real.png"  # Replace with the path to your uploaded PNG image
detect_stop_sign(uploaded_image_path)
'''

In [ ]:
'''
import cv2
import numpy as np
import pytesseract
from google.colab.patches import cv2_imshow  # Use this to display images in Colab

# Ensure Tesseract is properly set up
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'  # Update if necessary

def detect_road_sign(image_path):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Unable to load the image.")
        return

    # Resize the image for better processing
    image_resized = cv2.resize(image, (600, 400))

    # Convert to grayscale
    gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Perform edge detection using Canny
    edged = cv2.Canny(blurred, 50, 150)

    # Convert to HSV
    hsv = cv2.cvtColor(image_resized, cv2.COLOR_BGR2HSV)

    # Define color ranges
    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 70, 50])
    upper_red2 = np.array([180, 255, 255])
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])

    # Threshold the HSV image for colors
    mask_red = cv2.bitwise_or(cv2.inRange(hsv, lower_red1, upper_red1),
                              cv2.inRange(hsv, lower_red2, upper_red2))
    mask_white = cv2.inRange(hsv, lower_white, upper_white)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Clean up masks with morphological operations
    kernel = np.ones((3, 3), np.uint8)
    mask_red = cv2.morphologyEx(mask_red, cv2.MORPH_OPEN, kernel, iterations=2)
    mask_red = cv2.dilate(mask_red, kernel, iterations=1)
    mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_OPEN, kernel, iterations=2)
    mask_white = cv2.dilate(mask_white, kernel, iterations=1)
    mask_yellow = cv2.morphologyEx(mask_yellow, cv2.MORPH_OPEN, kernel, iterations=2)
    mask_yellow = cv2.dilate(mask_yellow, kernel, iterations=1)

    # Detect Stop Signs and No Parking Signs
    contours_red, _ = cv2.findContours(mask_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours_red:
        area = cv2.contourArea(contour)
        if area > 500:
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w) / h
            perimeter = cv2.arcLength(contour, True)
            circularity = 4 * np.pi * (area / (perimeter * perimeter))

            if circularity > 0.7:  # Circular shapes (for No Parking signs)
                roi = image_resized[y:y + h, x:x + w]
                roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                _, binary_roi = cv2.threshold(roi_gray, 50, 255, cv2.THRESH_BINARY_INV)
                custom_config = r'--psm 6'
                detected_text = pytesseract.image_to_string(binary_roi, config=custom_config)
                if 'P' in detected_text or "NO PARKING" in detected_text.upper():
                    cv2.rectangle(image_resized, (x, y), (x + w, y + h), (0, 255, 0), 3)
                    cv2.putText(image_resized, "No Parking Sign", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                elif len(approx) == 8 and 0.8 < aspect_ratio < 1.2:
                    cv2.drawContours(image_resized, [approx], 0, (0, 0, 255), 3)
                    cv2.putText(image_resized, "Stop Sign", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)



    # Detect Speed Limit Signs
    contours_white, _ = cv2.findContours(mask_white, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours_white:
        area = cv2.contourArea(contour)
        if area > 1000:
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w) / h
            if len(approx) == 4 and 0.5 < aspect_ratio < 1.5:
                cv2.drawContours(image_resized, [approx], 0, (255, 0, 0), 3)
                cv2.putText(image_resized, "Speed Limit Sign", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    # Detect Yield Signs
    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 800:
            epsilon = 0.04 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            if len(approx) == 3:
                x, y, w, h = cv2.boundingRect(approx)
                aspect_ratio = float(w) / h
                if 0.5 < aspect_ratio < 1.5:
                    cv2.drawContours(image_resized, [approx], 0, (0, 255, 255), 3)
                    cv2.putText(image_resized, "Yield Sign", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    # Detect Railroad Crossing Signs
    contours_yellow, _ = cv2.findContours(mask_yellow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours_yellow:
        area = cv2.contourArea(contour)
        if area > 1000:
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w) / h
            perimeter = cv2.arcLength(contour, True)
            circularity = 4 * np.pi * (area / (perimeter * perimeter))
            if 0.8 < aspect_ratio < 1.2 and circularity > 0.7:
                cv2.rectangle(image_resized, (x, y), (x + w, y + h), (0, 255, 0), 3)
                cv2.putText(image_resized, "Railroad Crossing", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Display the output
    cv2_imshow(image_resized)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Update the image path
image_path = "stopsign_real3.PNG"
detect_road_sign(image_path)
'''